This notebook is a wrapper script for the [Penn Phonetics Forced Aligner](https://babel.ling.upenn.edu/phonetics/old_website_2015/p2fa/index.html).

In [1]:
import os
from scipy.io import wavfile
import scipy.signal
from forced_aligner.align import main as p2fa
import numpy as np

In [ ]:
# Local paths, please update accordingly
git_path = '/path/to/git/kurteff2024_code/'
data_path = '/path/to/bids/dataset/'

In [ ]:
# Change these values accordingly
subj = "TCH14"
block = "B12"

In [ ]:
blockid = "_".join([subj,block])
w = os.path.join(data_path,f"sub-{subj}",blockid,"audio",f"{blockid}_mic.wav")
t = os.path.join(git_path,"preprocessing","events","transcripts",subj,blockid,f"{blockid}_mic.txt")
o = os.path.join(git_path,"preprocessing","events","textgrids",subj,blockid,f"{blockid}_mic.TextGrid")
m = os.path.join(git_path,"preprocessing","events","textgrids","forced_aligner","model")
fs, wav = wavfile.read(w)
if int(fs) != 11025:
    # The Penn Phonetics Forced Aligner only operates at specific sampling rates
    # The 11025 Hz model is a compromise between quality and speed and from
    # trial-and-error testing, it works well enough for our needs
    wav = signal.resample(wav, wav.shape[0]*11025/fs)
    wavfile.write(w,11025,wav)
# Run forced aligner
p2fa(w,t,o,m)

### Fix TextGrid rounding
P2FA has this bug where the TextGrids it create have these weird floating point errors that can cause some problems when trying to adjust boundaries in Praat's GUI. This next cell will attempt to automatically fix those, but it might require some troubleshooting (open the TextGrid in your preferred text editor and just check the `xmin/xmax`).

In [ ]:
# round / fix xmins
phone_tg_fpath = o
tab = "    "
fixed_float_grid = []
with open(phone_tg_fpath, 'r') as f:
    for i, line in enumerate(f):
        if 'xm' in line:
            if tab+tab+tab in line:
                indent = 19
            elif tab+tab in line:
                indent = 15
            elif tab in line:
                indent = 11
            else:
                indent = 7
            time = float(line[indent:])
            if 'xmin' in line and time <1:
                # Fix xmins here
                fixed_float_grid.append((line[:indent]+'0.0').replace('\n',''))
            else:
                # Round weird floating points here
                rounded_time = round(time,3) # to 3 decimal places
                fixed_float_grid.append((line[:indent]+str(rounded_time)).replace('\n',''))
        else:
            # For line that's not an xmin or xmax, simply add it to the fixed float grid unchanged
            fixed_float_grid.append(line.replace('\n',''))
# Save to file
np.savetxt(phone_tg_fpath,fixed_float_grid,fmt="%s")